In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import re
import scanpy as sc
from scipy.sparse import csr_matrix
import scvi
import torch
import seaborn as sns

In [3]:
filename = '../data_out/matrix_filtered_stim_after_filtering.h5ad'
adata_stim = sc.read(filename)

In [4]:
filename = '../data_out/matrix_filtered_nostim_after_filtering.h5ad'
adata_nostim = sc.read(filename)

In [6]:
adata = sc.concat([adata_stim, adata_nostim])

In [7]:
sc.pp.filter_genes(adata, min_cells = 10)

In [10]:
type(adata.X)

scipy.sparse._csr.csr_matrix

In [12]:
adata.write_h5ad('../data_out/matrix_combined.h5ad')

In [13]:
sc.pp.filter_genes(adata, min_cells = 100)

In [14]:
adata.layers['counts'] = adata.X.copy()

In [15]:
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [17]:
torch.set_float32_matmul_precision('medium')
scvi.settings.dl_num_workers = 19
scvi.settings.batch_size = 512


scvi.model.SCVI.setup_anndata(
    adata, layer = "counts",
    categorical_covariate_keys=["condition"],
    continuous_covariate_keys=['pct_counts_mt', 'total_counts', 'pct_counts_ribo']
)

model = scvi.model.SCVI(adata)
model.to_device('cuda')
model.train(batch_size=512)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training:   0%|          | 0/153 [00:00<?, ?it/s]

In [ ]:
adata.obsm['X_scVI'] = model.get_latent_representation()
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep = 'X_scVI')

In [ ]:
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution = 0.5)

In [ ]:
sc.pl.umap(adata, color = ['leiden', 'Sample'], frameon = False)

In [ ]:
adata.write_h5ad('integrated.h5ad')